In [ ]:
# send to database
# unique.to_sql('final_unique', engine)

In [1]:
# dependencies

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import datetime

db_password = "postgres"
db_user = "postgres"
db_name = "Youtube P3"
endpoint = 'youtube.cb1bticre0py.us-east-1.rds.amazonaws.com'

connection_string = f"postgresql://{db_user}:{db_password}@{endpoint}:5432/{db_name}"
engine = create_engine(connection_string)

og_data = pd.read_sql('SELECT * FROM final_unique;', con = engine)

In [11]:
# start here if you need to re-run instead of pulling from postgres in above cell

final_unique = og_data.copy()

In [12]:
## chose target_days for classification model

target_df = pd.DataFrame(final_unique[['video_id', 'trend_days']])

target_days = 8   #     <---- pick your target days here

# prints statement below
print(target_df[target_df.trend_days > target_days].count())
print(target_df[target_df.trend_days > 0].count())              
print(round((target_df[target_df.trend_days > target_days].count()) / (target_df[target_df.trend_days > 0].count()),4))     

#             ↓↓↓↓↓  videos above days threshold / total videos / % of dataset above days threshold

video_id      20508
trend_days    20508
dtype: int64
video_id      241112
trend_days    241112
dtype: int64
video_id      0.0851
trend_days    0.0851
dtype: float64


In [13]:
# create dataset with 50/50 > and < outcomes

over_target = final_unique[final_unique.trend_days > target_days]
under_target = final_unique[final_unique.trend_days <= target_days].sample(n=20508)  # <--- copy & paste top output of last cell

df = pd.concat([over_target, under_target]).reset_index(drop=True)

# add target to column to new 50/50 dataset

df['target'] = df.trend_days > target_days
df['target'] = df['target'].astype(int) # makes 1 or 0 for T or F

df.reset_index(drop=True, inplace=True)

print(over_target.shape)
print(under_target.shape)
print(df.shape)
print(" ")
df.columns

(20508, 34)
(20508, 34)
(41016, 35)
 


Index(['index', 'video_id', 'category', 'category_e', 'country',
       'publish_date', 'trending_date', 'publish_to_trend', 'publish_day',
       'publish_day_num', 'combined_trend_days', 'trend_days', 'views',
       'pt_views', 'wt_views', 'pt_views_rate', 'wt_views_rate', 'likes',
       'pt_likes', 'wt_likes', 'pt_likes_rate', 'wt_likes_rate', 'likes_ratio',
       'dislikes', 'pt_dislikes', 'wt_dislikes', 'pt_dislikes_rate',
       'wt_dislikes_rate', 'comments', 'pt_comments', 'wt_comments',
       'pt_comments_rate', 'wt_comments_rate', 'comments_ratio', 'target'],
      dtype='object')

In [6]:
# chose your features

X = df[[
    'category_e',
    'publish_to_trend',
    'publish_day_num',
    'pt_views',
    'pt_likes',
    'pt_dislikes',
    'pt_comments',
    'likes_ratio',
    'comments_ratio'
    ]].to_numpy()

y = df['target'].to_numpy()

In [ ]:
# if not scaling

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [8]:
# if scaling

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
sclr  = scaler.fit(X)
X_scaled = sclr.transform(X)

from sklearn.model_selection import train_test_split

X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X_scaled, y,random_state=1)

print(X_train_scaled.shape)
print(y_train.shape)

(30762, 9)
(30762,)
